In [3]:
import numpy as np
import matplotlib.pyplot as plt
import copy
from scipy import sparse

%matplotlib inline

# Лабораторная работа 5

### Таблично заданная функция (9.5(a))

| $x$  | $f(x)$   |
|------|----------|
| 0.   | 1.000000 |
| 0.25 | 0.989616 |
| 0.5  | 0.958851 |
| 0.75 | 0.908852 |
| 1.0  | 0.841471 |
| 1.25 | 0.759188 |
| 1.5  | 0.664997 |
| 1.75 | 0.562278 |
| 2.   | 0.454649 |



**Левые прямоугольники:**
$$
\int_{x_0}^{x_n} f(x)\,dx \approx \sum_{i=0}^{n-1} (x_{i+1}-x_i)\, f(x_i)
$$

**Правые прямоугольники:**
$$
\int_{x_0}^{x_n} f(x)\,dx \approx \sum_{i=0}^{n-1} (x_{i+1}-x_i)\, f(x_{i+1})
$$

**Средние прямоугольники:**
$$
\int_{x_0}^{x_n} f(x)\,dx \approx \sum_{i=0}^{n-1} (x_{i+1}-x_i)\, f\!\left(\frac{x_i+x_{i+1}}{2}\right)
$$


In [4]:
def left_rectangle_method(x, y):
    """Метод левых прямоугольников"""
    n = len(x) - 1
    integral = 0
    for i in range(n):
        h = x[i+1] - x[i]
        integral += h * y[i]  # значение в левой точке
    return integral

def right_rectangle_method(x, y):
    """Метод правых прямоугольников"""
    n = len(x) - 1
    integral = 0
    for i in range(n):
        h = x[i+1] - x[i]
        integral += h * y[i+1]  # значение в правой точке
    return integral

def middle_rectangle_method(x, y):
    """Метод средних прямоугольников"""
    n = len(x) - 1
    integral = 0
    for i in range(n):
        h = x[i+1] - x[i]
        y_mid = (y[i] + y[i+1]) / 2  # среднее значение на интервале
        integral += h * y_mid
    return integral

### Метод трапеций

$$ I_T = \sum_{k=0}^{n-1} \frac{h_k}{2} (f(x_{k+1}) + f(x_k)), \space h_k = x_{k+1} - x_k $$


In [5]:
def trapezoidal_method(x, y):
    """Метод трапеций"""
    n = len(x) - 1
    integral = 0
    for i in range(n):
        h = x[i+1] - x[i]
        integral += h * (y[i] + y[i+1]) / 2
    return integral

### Метод Симпсона

$$ I_S = \sum_{k=0}^{[n/2]} \frac{h_{2k}}{3} (f(x_{2k}) + 4f(x_{2k+1}) + f(x_{2k + 2}))$$

In [6]:
def simpson_method(x, y):
    """Метод Симпсона (параболическая формула)"""
    n = len(x) - 1
    if n < 2:
        raise ValueError("Для метода Симпсона нужно минимум 3 точки (2 интервала)")
    
    if n % 2 == 0:
        # Четное количество интервалов - составная формула
        h = (x[-1] - x[0]) / n
        integral = y[0] + y[-1]
        
        for i in range(1, n):
            if i % 2 == 0:
                integral += 2 * y[i]
            else:
                integral += 4 * y[i]
                
        return integral * h / 3
    else:
        # Нечетное количество интервалов
        integral = 0
        for i in range(0, n-1, 2):
            h = x[i+1] - x[i]
            integral += h/3 * (y[i] + 4*y[i+1] + y[i+2])
        return integral

### Метод Гаусса
Пусть $t_i$ — узлы, $w_i$ — веса формулы Гаусса на \([-1,1]\).
Тогда на \([a,b]\):

$$
\int_a^b f(x)\,dx \approx \frac{b-a}{2}\sum_{i=1}^{n} w_i\,
f\!\left(\frac{a+b}{2}+\frac{b-a}{2}\,t_i\right).
$$


In [7]:
def linear_interpolation(x_points, y_points, x):
    """Линейная интерполяция для метода Гаусса"""
    n = len(x_points)
    
    # Если x за пределами диапазона, используем крайние значения
    if x <= x_points[0]:
        return y_points[0]
    if x >= x_points[-1]:
        return y_points[-1]
    
    # Находим интервал
    for i in range(n-1):
        if x_points[i] <= x <= x_points[i+1]:
            # Линейная интерполяция
            t = (x - x_points[i]) / (x_points[i+1] - x_points[i])
            return y_points[i] * (1 - t) + y_points[i+1] * t
    
    return y_points[-1]  # на всякий случай

def gauss_method_2points(x, y):
    """Метод Гаусса с 2 точками (порядок 3)"""
    n_intervals = len(x) - 1
    integral = 0
    
    # Точки и веса для квадратуры Гаусса с 2 точками
    points = [-np.sqrt(1/3), np.sqrt(1/3)]
    weights = [1, 1]
    
    for i in range(n_intervals):
        a = x[i]
        b = x[i+1]
        
        # Преобразование к интервалу [-1, 1]
        def transform(t):
            return (b - a)/2 * t + (a + b)/2
        
        # Вычисление интеграла на интервале
        for point, weight in zip(points, weights):
            x_val = transform(point)
            y_val = linear_interpolation(x, y, x_val)
            integral += (b - a)/2 * weight * y_val
    
    return integral

def gauss_method_3points(x, y):
    """Метод Гаусса с 3 точками (порядок 5)"""
    n_intervals = len(x) - 1
    integral = 0
    
    # Точки и веса для квадратуры Гаусса с 3 точками
    points = [-np.sqrt(3/5), 0, np.sqrt(3/5)]
    weights = [5/9, 8/9, 5/9]
    
    for i in range(n_intervals):
        a = x[i]
        b = x[i+1]
        
        def transform(t):
            return (b - a)/2 * t + (a + b)/2
        
        for point, weight in zip(points, weights):
            x_val = transform(point)
            y_val = linear_interpolation(x, y, x_val)
            integral += (b - a)/2 * weight * y_val
    
    return integral

def gauss_method_4points(x, y):
    """Метод Гаусса с 4 точками (порядок 7)"""
    n_intervals = len(x) - 1
    integral = 0
    
    # Точки и веса для квадратуры Гаусса с 4 точками
    # Вычисляем точки Гаусса-Лежандра 4-го порядка
    sqrt30 = np.sqrt(30)
    t1 = np.sqrt((3 - 2*np.sqrt(6/5))/7)
    t2 = np.sqrt((3 + 2*np.sqrt(6/5))/7)
    
    points = [-t2, -t1, t1, t2]
    weights = [
        (18 - sqrt30) / 36,
        (18 + sqrt30) / 36,
        (18 + sqrt30) / 36,
        (18 - sqrt30) / 36
    ]
    
    for i in range(n_intervals):
        a = x[i]
        b = x[i+1]
        
        def transform(t):
            return (b - a)/2 * t + (a + b)/2
        
        for point, weight in zip(points, weights):
            x_val = transform(point)
            y_val = linear_interpolation(x, y, x_val)
            integral += (b - a)/2 * weight * y_val
    
    return integral


In [12]:
x = np.array([0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0])
y = np.array([1.000000, 0.989616, 0.958851, 0.908852, 0.841471, 
              0.759188, 0.664997, 0.562278, 0.454649])

print("=" * 70)
print("ЧИСЛЕННОЕ ИНТЕГРИРОВАНИЕ ТАБЛИЧНОЙ ФУНКЦИИ")
print(f"Интервал: [{x[0]}, {x[-1]}]")
print(f"Количество точек: {len(x)}")
print(f"Шаг: {x[1] - x[0]}")
print("=" * 70)

# Вычисление интегралов
print("\n1. МЕТОДЫ ПРЯМОУГОЛЬНИКОВ:")
print("-" * 40)
I_left = left_rectangle_method(x, y)
I_right = right_rectangle_method(x, y)
I_middle = middle_rectangle_method(x, y)

print(f"   Левые прямоугольники:    I = {I_left:.8f}")
print(f"   Правые прямоугольники:   I = {I_right:.8f}")
print(f"   Средние прямоугольники:  I = {I_middle:.8f}")

print("\n2. МЕТОД ТРАПЕЦИЙ:")
print("-" * 40)
I_trap = trapezoidal_method(x, y)
print(f"   Метод трапеций:          I = {I_trap:.8f}")

print("\n3. МЕТОД СИМПСОНА:")
print("-" * 40)
I_simp = simpson_method(x, y)
print(f"   Метод Симпсона:          I = {I_simp:.8f}")

print("\n4. МЕТОДЫ ГАУССА:")
print("-" * 40)
I_gauss2 = gauss_method_2points(x, y)
I_gauss3 = gauss_method_3points(x, y)
I_gauss4 = gauss_method_4points(x, y)

print(f"   Гаусс (2 точки):         I = {I_gauss2:.8f}")
print(f"   Гаусс (3 точки):         I = {I_gauss3:.8f}")
print(f"   Гаусс (4 точки):         I = {I_gauss4:.8f}")

# ============================================
# АНАЛИЗ И СРАВНЕНИЕ
# ============================================

print("\n" + "=" * 70)
print("СРАВНЕНИЕ МЕТОДОВ")
print("=" * 70)

# Создаем таблицу результатов
methods = {
    "Левые прямоугольники": I_left,
    "Правые прямоугольники": I_right,
    "Средние прямоугольники": I_middle,
    "Трапеций": I_trap,
    "Симпсона": I_simp,
    "Гаусса (2 точки)": I_gauss2,
    "Гаусса (3 точки)": I_gauss3,
    "Гаусса (4 точки)": I_gauss4
}

# Находим минимальное и максимальное значение
min_val = min(methods.values())
max_val = max(methods.values())
range_val = max_val - min_val

print(f"\nДиапазон значений: от {min_val:.8f} до {max_val:.8f}")
print(f"Размах: {range_val:.8f}")

print("\nОтносительные отклонения от среднего арифметического:")
print("-" * 50)

# Среднее арифметическое всех методов
avg_value = np.mean(list(methods.values()))

for name, value in methods.items():
    deviation = value - avg_value
    rel_deviation = (deviation / avg_value) * 100
    print(f"{name:25} | {value:.8f} | отклонение: {deviation:+.6f} ({rel_deviation:+.4f}%)")

ЧИСЛЕННОЕ ИНТЕГРИРОВАНИЕ ТАБЛИЧНОЙ ФУНКЦИИ
Интервал: [0.0, 2.0]
Количество точек: 9
Шаг: 0.25

1. МЕТОДЫ ПРЯМОУГОЛЬНИКОВ:
----------------------------------------
   Левые прямоугольники:    I = 1.67131325
   Правые прямоугольники:   I = 1.53497550
   Средние прямоугольники:  I = 1.60314437

2. МЕТОД ТРАПЕЦИЙ:
----------------------------------------
   Метод трапеций:          I = 1.60314437

3. МЕТОД СИМПСОНА:
----------------------------------------
   Метод Симпсона:          I = 1.60541858

4. МЕТОДЫ ГАУССА:
----------------------------------------
   Гаусс (2 точки):         I = 1.60314437
   Гаусс (3 точки):         I = 1.60314438
   Гаусс (4 точки):         I = 1.60314438

СРАВНЕНИЕ МЕТОДОВ

Диапазон значений: от 1.53497550 до 1.67131325
Размах: 0.13633775

Относительные отклонения от среднего арифметического:
--------------------------------------------------
Левые прямоугольники      | 1.67131325 | отклонение: +0.067885 (+4.2337%)
Правые прямоугольники     | 1.53497550 | откл